### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [30]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [31]:
articles_data = []

for page_num in range(1, 300):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/lao-dong-tien-luong/t1/11/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))
    

#### 2.3 Export csv

In [32]:
# Tạo DataFrame từ danh sách dữ liệu
df_laodong_tienthuong = pd.DataFrame(articles_data)
df_laodong_tienthuong.head(5)

,Name,Time,Question,Situation,Answer
0,Trịnh Minh Tâm,09:36 07/08/2024,Từng bị kỷ luật có được đề nghị xét tặng Huân ...,"Ông A là Phó Giám đốc Sở, nhận thông báo nghỉ ...",Bộ Nội vụ trả lời vấn đề này như sau:Điểm b Kh...
1,Võ Thị Thu Lan,07:53 07/08/2024,Các chi nhánh có phải ban hành nội quy lao độn...,"Tôi xin hỏi, tổng công ty TNHH MTV do Nhà nước...",Bộ Lao động – Thương binh và Xã hội trả lời vấ...
2,Cao Linh,08:34 06/08/2024,Trường hợp nào được trợ cấp chuyển vùng?,"Ông Nguyễn Văn A là viên chức, công tác tại vù...",Bộ Nội vụ trả lời vấn đề này như sau:Nghị định...
3,Công ty CP S,08:25 06/08/2024,Điều kiện gửi văn bản ký số thay văn bản giấy,Công ty chúng tôi thường gửi văn bản cho cơ qu...,Bộ Nội vụ trả lời vấn đề này như sau:Để đảm bả...
4,Thái Thị Hoài,09:05 05/08/2024,Căn cứ xét tặng Bằng khen của Thủ tướng,Tôi là giáo viên THPT. Từ năm 2019 đến năm 202...,Bộ Nội vụ trả lời vấn đề này như sau:Điểm b Kh...


In [33]:

# Lưu DataFrame vào file CSV
csv_file = '../data/laodong_tienthuong_data.csv'
df_laodong_tienthuong.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../data/laodong_tienthuong_data.csv


In [2]:
df_laodong_tienthuong=pd.read_csv("../data/laodong_tienthuong_data.csv",encoding='utf-8')

In [3]:
df_laodong_tienthuong.head(5)

,Unnamed: 0,Name,Time,Question,Situation,Answer
0,0,Trịnh Minh Tâm,09:36 07/08/2024,Từng bị kỷ luật có được đề nghị xét tặng Huân ...,"Ông A là Phó Giám đốc Sở, nhận thông báo nghỉ ...",Bộ Nội vụ trả lời vấn đề này như sau:Điểm b Kh...
1,1,Võ Thị Thu Lan,07:53 07/08/2024,Các chi nhánh có phải ban hành nội quy lao độn...,"Tôi xin hỏi, tổng công ty TNHH MTV do Nhà nước...",Bộ Lao động – Thương binh và Xã hội trả lời vấ...
2,2,Cao Linh,08:34 06/08/2024,Trường hợp nào được trợ cấp chuyển vùng?,"Ông Nguyễn Văn A là viên chức, công tác tại vù...",Bộ Nội vụ trả lời vấn đề này như sau:Nghị định...
3,3,Công ty CP S,08:25 06/08/2024,Điều kiện gửi văn bản ký số thay văn bản giấy,Công ty chúng tôi thường gửi văn bản cho cơ qu...,Bộ Nội vụ trả lời vấn đề này như sau:Để đảm bả...
4,4,Thái Thị Hoài,09:05 05/08/2024,Căn cứ xét tặng Bằng khen của Thủ tướng,Tôi là giáo viên THPT. Từ năm 2019 đến năm 202...,Bộ Nội vụ trả lời vấn đề này như sau:Điểm b Kh...


#### 2.4.Export PDF

> Function

In [4]:

df_selected = df_laodong_tienthuong[['Question', 'Situation', 'Answer']]

# Tạo lớp kế thừa từ FPDF để tạo PDF
class PDF(FPDF):
    def header(self):
        self.set_font('DejaVuSans', '', 12)
        self.cell(0, 10, 'Danh sách câu hỏi và trả lời', 0, 1, 'C')
    
    def chapter_title(self, title):
        self.set_font('DejaVuSans-Bold', '', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(10)
    
    def chapter_body(self, body):
        self.set_font('DejaVuSans', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()
    
    def add_question(self, question, situation, answer):
        self.add_page()
        self.chapter_title('Question:')
        self.chapter_body(question)
        self.chapter_title('Situation:')
        self.chapter_body(situation)
        self.chapter_title('Answer:')
        self.chapter_body(answer)

> Execute funtion

In [5]:

# Tạo đối tượng PDF
pdf = PDF()
pdf.add_font('DejaVuSans', '', 'DejaVuSans.ttf', uni=True)  # Thêm font Unicode
pdf.add_font('DejaVuSans-Bold', '', 'dejavu-sans-bold.ttf', uni=True)  # Thêm font Unicode
pdf.set_left_margin(10)
pdf.set_right_margin(10)

# Thêm dữ liệu vào PDF
for index, row in df_selected.iterrows():
    pdf.add_question(row['Question'], row['Situation'], row['Answer'])

# Lưu file PDF
pdf.output('../data/laodong_tienthuong.pdf', 'F')


''